In [1]:
import os

db_dir = os.environ["DATA"] + "PatImgXAI_data/db1.0.0/"
test_dataset_filename = "rowcircles_test.csv"

model_dir = os.environ["DATA"] + "models/db1.0.0/rowcircles/"



In [2]:
# Probability to generate a geometrical shape at each position in the grid
SHAPE_PROB = 0.5

# Define available shapes
SHAPES = ['circle', 'square', 'triangle']
COLORS  = ["#F86C62", "#7AB0CD", "#F4D67B", "#87C09C"]

In [3]:
from xaipatimg.datagen.dbimg import load_db

db = load_db(db_dir)

In [3]:
import numpy as np
from xaipatimg.ml.xai import generate_shap_resnet18

generate_shap_resnet18(db_dir, test_dataset_filename, model_dir, "cuda:0", n_jobs=20, dataset_size=20, masker="ndarray")

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading dataset content for rowcircles_test.csv


100%|██████████| 20/20 [00:00<00:00, 171.41it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Computing shap values


 99%|█████████▉| 9942/9998 [00:13<00:00, 542.36it/s]
10042it [00:13, 535.28it/s]                         
 99%|█████████▉| 9942/9998 [00:13<00:00, 509.87it/s]
10042it [00:13, 505.82it/s]                         
 99%|█████████▉| 9942/9998 [00:13<00:00, 508.31it/s]
10042it [00:13, 504.37it/s]                         
100%|█████████▉| 9992/9998 [00:13<00:00, 473.48it/s]
10042it [00:13, 475.38it/s]                         
100%|█████████▉| 9992/9998 [00:13<00:00, 493.12it/s]
10042it [00:13, 494.44it/s]                         
100%|█████████▉| 9992/9998 [00:13<00:00, 510.81it/s]
10048it [00:13, 502.10it/s]                         
100%|█████████▉| 9992/9998 [00:13<00:00, 512.97it/s]
10048it [00:13, 509.02it/s]                         
100%|█████████▉| 9992/9998 [00:13<00:00, 501.73it/s]
10048it [00:13, 500.00it/s]                         
 99%|█████████▉| 9942/9998 [00:13<00:00, 505.75it/s]
10042it [00:13, 502.36it/s]                         
100%|█████████▉| 9992/9998 [00:13<00:00, 510.0

Generating shap images


100%|██████████| 20/20 [00:02<00:00,  7.53it/s]


Counterfactuals

In [5]:
import numpy as np

def extract_rows_with_only_circles(img_content):
    circles_counter = np.zeros(6,)
    non_circles_counter = np.zeros(6,)

    for c in img_content:
        if c["shape"] == "circle":
            circles_counter[c["pos"][1]] += 1
        else:
            non_circles_counter[c["pos"][1]] += 1

    return np.logical_and(circles_counter >= 1, non_circles_counter == 0)

def exist_row_with_only_circles(img_content):
    return np.sum(extract_rows_with_only_circles(img_content)) >= 1

In [6]:
from xaipatimg.datagen.utils import gen_rand_sym, PatImgObj
import numpy as np
COLORS  = ["#F86C62", "#7AB0CD", "#F4D67B", "#87C09C"]

def rowcircles_counterfactuals(img_entry, is_pos, nb_cf):

    output_cf = []

    # Case we are searching for negative counterfactuals
    if is_pos:
        # Extracting indices of lines with only circles
        rows_with_only_circles = np.nonzero(extract_rows_with_only_circles(img_entry["content"]))[0]

        # Iterating over all counterfactuals to generate.
        for _ in range(nb_cf):

            patimgobj = PatImgObj(img_entry)

            # Randomly setting a non circle symbol in every row that only contains circles
            for j_row in rows_with_only_circles:
                patimgobj.set_symbol(posx=np.random.choice(np.arange(img_entry["division"][0])),
                                     posy=j_row,
                                     value=gen_rand_sym(shapes=["square", "triangle"], colors=COLORS))

            output_cf.append(patimgobj.get_img_dict())

    # Case we are looking for a positive counterfactual
    else:
        patimgobj = PatImgObj(img_entry)

        # Creating the list of lines which are not empty in a random order
        non_empty_lines_rnd = np.setdiff1d(np.arange(img_entry["division"][1]), patimgobj.get_empty_lines())
        np.random.shuffle(non_empty_lines_rnd)

        for idx in range(min(nb_cf, len(non_empty_lines_rnd))):
            patimgobj = PatImgObj(img_entry)
            patimgobj.change_shapes_of_line(non_empty_lines_rnd[idx], "circle")
            output_cf.append(patimgobj.get_img_dict())

    return output_cf


In [ ]:
from xaipatimg.ml.xai import generate_counterfactuals_resnet18

generate_counterfactuals_resnet18(db_dir, test_dataset_filename, model_dir, rowcircles_counterfactuals, 5, None, 1, None)

Loading dataset content for rowcircles_test.csv


100%|██████████| 1000/1000 [00:16<00:00, 62.43it/s]
Using cache found in /home/docker/.cache/torch/hub/pytorch_vision_v0.10.0
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/docker/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [4]:
from xaipatimg.ml.xai import generate_counterfactuals_resnet18_random_approach

generate_counterfactuals_resnet18_random_approach(db_dir, test_dataset_filename, model_dir, SHAPES, COLORS, 1-SHAPE_PROB, max_depth=5,
                                                  nb_tries_per_depth=2000, device="cuda:0", n_jobs=1, dataset_size=100)

/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading dataset content for rowcircles_test.csv


100%|██████████| 100/100 [00:00<00:00, 165.42it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Generating counterfactual images


  0%|          | 0/100 [00:00<?, ?it/s]

...

Loading dataset content for /tmp/tmpy302jmdh/dataset.csv



100%|██████████| 2000/2000 [00:12<00:00, 164.92it/s]
Using cache found in /home/jleguy/.cache/torch/hub/pytorch_vision_v0.10.0
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jleguy/anaconda3/envs/Pytorch_exp/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
 23%|██▎       | 23/100 [19:56<54:36, 42.55s/it]  

Loading dataset content for /tmp/tmpcf30e85m/dataset.csv



 57%|█████▋    | 1146/2000 [00:11<00:08, 101.37it/s]


KeyboardInterrupt: 